# Smart Buildings HVAC Simulation Demo

This notebook demonstrates how to:
1. Load a building config and weather
2. Generate a simulation environment
3. Run baseline policies
4. Visualize results

In [ ]:
from pathlib import Path
import numpy as np

from sbsim.smart_control.utils.scenario_generator import (
    load_scenario_from_parts,
    generate_scenario_from_config,
    get_env,
    print_action_spec,
    SimulationTracker,
)

# List available buildings
buildings_dir = Path("buildings")
print("Available buildings:")
for f in sorted(buildings_dir.glob("*.yaml")):
    print(f"  {f.name}")

In [ ]:
# List available weathers
weathers_dir = Path("weathers")
print("Available weathers:")
for f in sorted(weathers_dir.glob("*.yaml")):
    print(f"  {f.stem}")

In [ ]:
# SELECT BUILDING AND WEATHER
BUILDING = "buildings/building_0001.yaml"
WEATHER = "temperate"  # or path to weather yaml

In [ ]:
# Generate scenario
print(f"Loading {BUILDING} with {WEATHER} weather...")
scenario_config = load_scenario_from_parts(BUILDING, WEATHER, output_base_dir="demo_output")
result = generate_scenario_from_config(scenario_config)

print(f"Generated {result['num_rooms']} rooms with {result['num_ahus']} AHUs")
print(f"Output directory: {result['output_dir']}")

In [ ]:
# Create environment
env = get_env(result)
print(f"Reward weights: {env.reward_function.weights}")

# Create tracker for visualization
tracker = SimulationTracker(env, vmin=280, vmax=310)

In [ ]:
# Print action specification
example_action = print_action_spec(env)

## Baseline Policies

- **all_off**: All HVAC systems off
- **all_on**: All systems on at max capacity
- **bang_bang**: Simple thermostat control

In [ ]:
def get_action_indices(env):
    """Get indices for each action type in continuous_action."""
    temp_indices = []
    pressure_indices = []
    water_temp_indices = []
    
    idx = 0
    for device, action_name in env._device_action_tuples:
        if 'run_command' in action_name:
            continue  # discrete
        if 'supply_air_temperature' in action_name:
            temp_indices.append(idx)
        elif 'supply_water_setpoint' in action_name:
            water_temp_indices.append(idx)
        else:  # pressure setpoints
            pressure_indices.append(idx)
        idx += 1
    
    return temp_indices, pressure_indices, water_temp_indices

def make_action(env, discrete_value: int, temp_value: float, pressure_value: float, water_value: float):
    """Create action dict with proper values per action type."""
    n_discrete = env.action_spec()['discrete_action'].shape[0]
    n_continuous = env.action_spec()['continuous_action'].shape[0]
    
    temp_idx, pressure_idx, water_idx = get_action_indices(env)
    
    continuous = [0.0] * n_continuous
    for i in temp_idx:
        continuous[i] = temp_value
    for i in pressure_idx:
        continuous[i] = pressure_value
    for i in water_idx:
        continuous[i] = water_value
    
    return {
        'discrete_action': [discrete_value] * n_discrete,
        'continuous_action': continuous,
    }

def get_avg_zone_temp(env):
    """Get average zone temperature from environment."""
    temps = [vav.zone_air_temperature for vav in env.building.simulator._hvac._vavs.values()]
    return np.mean(temps)

def get_current_setpoints(env):
    """Get current (heating, cooling) setpoints based on schedule."""
    schedule = env.building.simulator._hvac.schedule
    timestamp = env.building.current_timestamp
    return schedule.get_temperature_window(timestamp)

In [ ]:
def run_baseline(env, tracker, policy: str, steps: int = 100):
    """Run a baseline policy."""
    tracker.reset()
    
    for _ in range(steps):
        if policy == "all_off":
            action = make_action(env, discrete_value=0, temp_value=-1, pressure_value=-1, water_value=-1)
        
        elif policy == "all_on":
            action = make_action(env, discrete_value=1, temp_value=-1, pressure_value=1, water_value=1)
        
        elif policy == "bang_bang":
            zone_temp = get_avg_zone_temp(env)
            heating_setpoint, cooling_setpoint = get_current_setpoints(env)
            
            if zone_temp < heating_setpoint:
                action = make_action(env, discrete_value=1, temp_value=1, pressure_value=1, water_value=1)
            elif zone_temp > cooling_setpoint:
                action = make_action(env, discrete_value=1, temp_value=-1, pressure_value=1, water_value=1)
            else:
                action = make_action(env, discrete_value=0, temp_value=-1, pressure_value=-1, water_value=-1)
        
        tracker.step(action)

In [ ]:
# Run a single policy (bang_bang is usually best)
STEPS = 12 * 24 * 2  # 2 days at 5-min intervals

print("Running bang_bang policy...")
run_baseline(env, tracker, "bang_bang", steps=STEPS)
print(f"Total reward: {sum(tracker.rewards):.2f}")

In [ ]:
# Show overview plot with AHU coloring
tracker.plot_overview(color_by_ahu=True)

In [ ]:
# Show reward breakdown
tracker.plot_rewards()

In [ ]:
# Show AHU details
tracker.plot_ahu()

In [ ]:
# Render a specific timestep
tracker.render(step=100)

## Compare All Baselines

In [ ]:
# Compare all three baselines
results = {}
for policy in ["all_off", "all_on", "bang_bang"]:
    print(f"\nRunning {policy}...")
    run_baseline(env, tracker, policy, steps=STEPS)
    results[policy] = sum(tracker.rewards)
    print(f"  Reward: {results[policy]:.2f}")

print(f"\n{'='*40}")
print("Summary:")
for name, reward in sorted(results.items(), key=lambda x: -x[1]):
    print(f"  {name}: {reward:.2f}")

## Create Animation

In [ ]:
# Re-run best policy and create animation
run_baseline(env, tracker, "bang_bang", steps=STEPS)

# Save as GIF (or mp4)
tracker.create_video("demo_simulation.gif", fps=10)
print("Saved demo_simulation.gif")

In [ ]:
# Display the GIF
from IPython.display import Image
Image(filename="demo_simulation.gif")